In [ ]:
import os
os.chdir('/opt/ml/train')
print(os.getcwd())

In [ ]:
!pip install torch_optimizer

In [ ]:
!pip install seaborn
!pip install efficientnet_pytorch
!pip install fastai==1.0.61
!pip install git+https://github.com/ildoonet/pytorch-randaugment
!pip install natsort

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
import glob
import numpy as np
import pickle
#from tensorboardX import SummaryWriter
import time
import os,shutil,json
import argparse
#import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
#import natsort
#import easydict
import PIL
from PIL import Image as Img
#from skimage import io, transform
import random
import math
import torch.utils.data
import pandas as pd
import seaborn as sns
from sklearn.model_selection import KFold
from tqdm import tqdm
#import imutils
import zipfile
#from efficientnet_pytorch import EfficientNet
from RandAugment import RandAugment
# from fastai import *
# from fastai.vision import *
import torch_optimizer as optim

In [ ]:
!ls

In [ ]:
load_df = pd.read_csv('train.csv')
load_df.head()

In [ ]:
print(max(load_df.age))
print(min(load_df.age))
sns.set_theme()
sns.distplot(load_df['age'], kde=True, bins=30)

In [ ]:
li = []
li2 = []
for idx, row in load_df.iterrows():
  path = glob.glob('images/'+row['path']+'/*')
  li.append(path)
  li2.append([os.path.basename(x).split('.')[0][:-1] for x in path])
flatten_li = list(np.concatenate(li).flat)
flatten_li2 = list(np.concatenate(li2).flat)
#print(flatten_li2)

In [ ]:
train_df = []
for row in load_df.itertuples(index=False):
  train_df.extend([list(row)]*7)
train_df = pd.DataFrame(train_df, columns=load_df.columns)
train_df.head(20)

In [ ]:
train_df['path']=flatten_li
train_df['mask']=flatten_li2
train_df.head()

In [ ]:
del train_df['id']
del train_df['race']
train_df.head()

In [ ]:
def age2idx(age):
  if age < 30:
    return 0
  elif age >=60:
    return 2
  else:
    return 1

In [ ]:
gender = train_df['gender'].unique()
#age = train_df['age'].unique()
mask = train_df['mask'].unique()
#print(gender, age, mask)
gender2idx = {o:i for i,o in enumerate(gender)}
#age2idx = {o:i for i,o in enumerate(age)}
mask2idx = {o:i for i,o in enumerate(mask)}

train_df['gender'] = train_df['gender'].apply(lambda x: gender2idx[x])
train_df['age'] = train_df['age'].apply(lambda x: age2idx(x))
train_df['mask'] = train_df['mask'].apply(lambda x: mask2idx[x])

train_df.head()

In [ ]:
sns.set_theme()
sns.distplot(train_df['mask'], kde=True, bins=30)

In [ ]:
append_df = []
for row in train_df.itertuples(index=False):
  if row.mask != 0:
    append_df.extend([list(row)]*1)
append_df = pd.DataFrame(append_df, columns=train_df.columns)
append_df.head()

In [ ]:
train_df = pd.concat([train_df, append_df])

In [ ]:
sns.set_theme()
sns.distplot(train_df['mask'], kde=True, bins=30)

In [ ]:
class MultiTaskDataset(torch.utils.data.Dataset):
    def __init__(self, df, tf):
        self.path = list(df['path'])
        self.gender = list(df['gender'])

        self.transform = transforms.Compose([
            # transforms.Resize((224, 224)),
            transforms.CenterCrop((350,256)),            
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
            ])
        if tf:
            self.transform.transforms.insert(0,RandAugment(2,9))

    def __len__(self): return len(self.path)

    def __getitem__(self,idx):
        #dealing with the image
        im = PIL.Image.open(self.path[idx]).convert('RGB')
        im = self.transform(im)


        #dealing with the labels
        gender = torch.tensor(int(self.gender[idx]), dtype=torch.int64)
        sample = {
            'image':im,
            'label':gender
        }
        
        return sample

In [ ]:
class MultiLabelResnext(nn.Module):
    def __init__(self):
        super(MultiLabelResnext, self).__init__()
        self.resnext = models.resnext101_32x8d(pretrained=True)
        #self.resnext = models.resnext50_32x4d(pretrained=True)
        self.drop = nn.Dropout(p=0.7)
        self.FC = nn.Linear(1000, 3)

    def forward(self, x):
       
        # x = F.silu(self.conv2d(x))

        # efficientnet b7
        # 원래는 efficientnet과 동일하게 swish를 넣으려고 했으나
        # 별 차이 없는 듯 해서 그냥 silu 적용
        x = F.gelu(self.resnext(x))

        x = self.drop(x)
        x = self.FC(x)
        return x
model = MultiLabelResnext()
model

In [ ]:
device = torch.device("cuda:0")

In [ ]:
def accuracy(log_pred, y_true):
    y_pred = np.argmax(log_pred)
    return np.mean(y_pred == y_true)

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
#scaler = torch.cuda.amp.GradScaler()
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(train_df),1):
    print(f'[fold: {fold_index}]')
    
    # cuda cache 초기화
    torch.cuda.empty_cache()

    #train fold, validation fold 분할
    train_answer = train_df.iloc[trn_idx]
    test_answer  = train_df.iloc[val_idx]
    
    #Dataset 정의
    train_dataset = MultiTaskDataset(train_answer, tf=True)
    valid_dataset = MultiTaskDataset(test_answer, tf=False)
    
    #DataLoader 정의
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size = 64,
        shuffle = True,
        num_workers = 2
    )
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size = 32,
        shuffle = False,
        num_workers = 2
    )

    # 모델 선언
    #model = MultiLabelEfficientnet()
    model = MultiLabelResnext()
    model.to(device)# gpu에 모델 할당
    
    # 훈련 옵션 설정
    #optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
    optimizer = optim.RAdam(model.parameters(), lr = 1e-3)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size = 5,
                                                gamma = 0.9)
    criterion = torch.nn.CrossEntropyLoss()
    
    # 훈련 시작
    valid_acc_max = 0
    valid_loss_min = float("inf")
    for epoch in range(30):
        # 1개 epoch 훈련
        train_acc_list = []
        train_loss_list = []
        with tqdm(train_data_loader,#train_data_loader를 iterative하게 반환
                total=train_data_loader.__len__(), # train_data_loader의 크기
                unit="batch") as train_bar:# 한번 반환하는 sample의 단위는 "batch"
            for sample in train_bar:
                train_bar.set_description(f"Train Epoch {epoch}")
                
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                # tensor를 gpu에 올리기 
                images = images.to(device)
                labels = labels.to(device)
            
                # 모델의 dropoupt, batchnormalization를 train 모드로 설정
                model.train()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.set_grad_enabled(True):
                    # 모델 예측
                    probs  = model(images)
                    # loss 계산
                    loss = criterion(probs, labels)
                    # 중간 노드의 gradient로
                    # backpropagation을 적용하여
                    # gradient 계산
                    loss.backward()
                    # weight 갱신
                    optimizer.step()
                    
                    # train accuracy 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    #preds = probs > 0.5
                    batch_acc = accuracy(probs, labels)
                    train_acc_list.append(batch_acc)
                    train_acc = np.mean(train_acc_list)
                    train_loss_list.append(loss.item())
                    train_loss = np.mean(train_loss_list)
                # 현재 progress bar에 현재 미니배치의 loss 결과 출력
                train_bar.set_postfix(train_loss= train_loss,
                                      train_acc = train_acc)
                
        # 1개 epoch학습 후 Validation 점수 계산
        valid_acc_list = []
        valid_loss_list = []
        with tqdm(valid_data_loader,
                total=valid_data_loader.__len__(),
                unit="batch") as valid_bar:
            for sample in valid_bar:
                valid_bar.set_description(f"Valid Epoch {epoch}")
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                images = images.to(device)
                labels = labels.to(device)
                
                # 모델의 dropoupt, batchnormalization를 eval모드로 설정
                model.eval()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.no_grad():
                    # validation loss만을 계산
                    probs  = model(images)
                    valid_loss = criterion(probs, labels)

                    # train accuracy 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
#                     preds = probs > 0.5
                    batch_acc = accuracy(probs, labels)
                    valid_acc_list.append(batch_acc)
                    valid_loss_list.append(valid_loss.item())
                valid_acc = np.mean(valid_acc_list)
                valid_loss = np.mean(valid_loss_list)
                valid_bar.set_postfix(valid_loss = valid_loss,
                                      valid_acc = valid_acc)
                
        # Learning rate 조절
        lr_scheduler.step()
        
        # 모델 저장
        if valid_loss_min > valid_loss:
            valid_loss_min = valid_loss
            best_model = model
            
    # 폴드별로 가장 좋은 모델 저장
    MODEL = "resnext101_gender"
    # 모델을 저장할 구글 드라이브 경로
    path = "/opt/ml/pstage_1/"
    torch.save(best_model, f'{path}{MODEL}_{fold_index}_{valid_loss:2.4f}.pth')
    best_models.append(best_model)

In [ ]:
!nvidia-smi